In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', number_of_syllables=True)
data.generate_data('letters_word_accetuation_num_train',
                   'letters_word_accetuation_num_test',
                   'letters_word_accetuation_num_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [12]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_other_features_train[pos])
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[2, 5]


In [6]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20




conv_input_shape=(23, 36)
othr_input = (141, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=3)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch),
                              verbose=2,
                              workers=4)

Epoch 1/800
15s - loss: 0.2134 - actual_accuracy: 0.4034 - val_loss: 0.1514 - val_actual_accuracy: 0.6208
Epoch 2/800
16s - loss: 0.1536 - actual_accuracy: 0.5592 - val_loss: 0.1309 - val_actual_accuracy: 0.6357
Epoch 3/800
17s - loss: 0.1377 - actual_accuracy: 0.6204 - val_loss: 0.1211 - val_actual_accuracy: 0.6860
Epoch 4/800
17s - loss: 0.1272 - actual_accuracy: 0.6638 - val_loss: 0.1064 - val_actual_accuracy: 0.7477
Epoch 5/800
17s - loss: 0.1195 - actual_accuracy: 0.6943 - val_loss: 0.1063 - val_actual_accuracy: 0.7558
Epoch 6/800
18s - loss: 0.1134 - actual_accuracy: 0.7124 - val_loss: 0.0974 - val_actual_accuracy: 0.7684
Epoch 7/800
19s - loss: 0.1081 - actual_accuracy: 0.7283 - val_loss: 0.0915 - val_actual_accuracy: 0.7818
Epoch 8/800
18s - loss: 0.1030 - actual_accuracy: 0.7416 - val_loss: 0.0929 - val_actual_accuracy: 0.7899
Epoch 9/800
17s - loss: 0.0971 - actual_accuracy: 0.7565 - val_loss: 0.0902 - val_actual_accuracy: 0.8014
Epoch 10/800
17s - loss: 0.0958 - actual_accur

20s - loss: 0.0233 - actual_accuracy: 0.9462 - val_loss: 0.0826 - val_actual_accuracy: 0.8857
Epoch 79/800
21s - loss: 0.0232 - actual_accuracy: 0.9467 - val_loss: 0.0798 - val_actual_accuracy: 0.8923
Epoch 80/800
19s - loss: 0.0241 - actual_accuracy: 0.9474 - val_loss: 0.0841 - val_actual_accuracy: 0.8869
Epoch 81/800
20s - loss: 0.0236 - actual_accuracy: 0.9451 - val_loss: 0.0799 - val_actual_accuracy: 0.8926
Epoch 82/800
19s - loss: 0.0221 - actual_accuracy: 0.9500 - val_loss: 0.0770 - val_actual_accuracy: 0.8969
Epoch 83/800
20s - loss: 0.0216 - actual_accuracy: 0.9517 - val_loss: 0.0860 - val_actual_accuracy: 0.8907
Epoch 84/800
20s - loss: 0.0228 - actual_accuracy: 0.9473 - val_loss: 0.0757 - val_actual_accuracy: 0.9064
Epoch 85/800
20s - loss: 0.0214 - actual_accuracy: 0.9518 - val_loss: 0.0820 - val_actual_accuracy: 0.8953
Epoch 86/800
20s - loss: 0.0220 - actual_accuracy: 0.9487 - val_loss: 0.0889 - val_actual_accuracy: 0.8884
Epoch 87/800
19s - loss: 0.0207 - actual_accuracy:

21s - loss: 0.0134 - actual_accuracy: 0.9704 - val_loss: 0.1102 - val_actual_accuracy: 0.8846
Epoch 156/800
21s - loss: 0.0124 - actual_accuracy: 0.9725 - val_loss: 0.1179 - val_actual_accuracy: 0.8949
Epoch 157/800
22s - loss: 0.0115 - actual_accuracy: 0.9740 - val_loss: 0.1072 - val_actual_accuracy: 0.8946
Epoch 158/800
21s - loss: 0.0123 - actual_accuracy: 0.9735 - val_loss: 0.1094 - val_actual_accuracy: 0.8984
Epoch 159/800
21s - loss: 0.0125 - actual_accuracy: 0.9737 - val_loss: 0.1103 - val_actual_accuracy: 0.8957
Epoch 160/800
21s - loss: 0.0126 - actual_accuracy: 0.9735 - val_loss: 0.0990 - val_actual_accuracy: 0.8965
Epoch 161/800
21s - loss: 0.0126 - actual_accuracy: 0.9716 - val_loss: 0.0944 - val_actual_accuracy: 0.9026
Epoch 162/800
21s - loss: 0.0121 - actual_accuracy: 0.9739 - val_loss: 0.0912 - val_actual_accuracy: 0.9064
Epoch 163/800
21s - loss: 0.0127 - actual_accuracy: 0.9729 - val_loss: 0.1056 - val_actual_accuracy: 0.8923
Epoch 164/800
21s - loss: 0.0125 - actual_

20s - loss: 0.0084 - actual_accuracy: 0.9809 - val_loss: 0.1323 - val_actual_accuracy: 0.8882
Epoch 232/800
20s - loss: 0.0082 - actual_accuracy: 0.9826 - val_loss: 0.1145 - val_actual_accuracy: 0.8965
Epoch 233/800
21s - loss: 0.0083 - actual_accuracy: 0.9821 - val_loss: 0.1191 - val_actual_accuracy: 0.8953
Epoch 234/800
21s - loss: 0.0089 - actual_accuracy: 0.9803 - val_loss: 0.1240 - val_actual_accuracy: 0.9045
Epoch 235/800
20s - loss: 0.0083 - actual_accuracy: 0.9814 - val_loss: 0.1190 - val_actual_accuracy: 0.9015
Epoch 236/800
21s - loss: 0.0075 - actual_accuracy: 0.9831 - val_loss: 0.1022 - val_actual_accuracy: 0.9049
Epoch 237/800
21s - loss: 0.0083 - actual_accuracy: 0.9820 - val_loss: 0.1071 - val_actual_accuracy: 0.8969
Epoch 238/800
22s - loss: 0.0086 - actual_accuracy: 0.9819 - val_loss: 0.1127 - val_actual_accuracy: 0.9095
Epoch 239/800
20s - loss: 0.0078 - actual_accuracy: 0.9837 - val_loss: 0.1000 - val_actual_accuracy: 0.9110
Epoch 240/800
21s - loss: 0.0081 - actual_

21s - loss: 0.0063 - actual_accuracy: 0.9861 - val_loss: 0.1203 - val_actual_accuracy: 0.8923
Epoch 308/800
21s - loss: 0.0061 - actual_accuracy: 0.9877 - val_loss: 0.1397 - val_actual_accuracy: 0.8947
Epoch 309/800
21s - loss: 0.0066 - actual_accuracy: 0.9854 - val_loss: 0.1209 - val_actual_accuracy: 0.8857
Epoch 310/800
21s - loss: 0.0064 - actual_accuracy: 0.9858 - val_loss: 0.1230 - val_actual_accuracy: 0.9011
Epoch 311/800
20s - loss: 0.0060 - actual_accuracy: 0.9872 - val_loss: 0.1274 - val_actual_accuracy: 0.8965
Epoch 312/800
20s - loss: 0.0065 - actual_accuracy: 0.9845 - val_loss: 0.1249 - val_actual_accuracy: 0.9045
Epoch 313/800
20s - loss: 0.0061 - actual_accuracy: 0.9868 - val_loss: 0.1105 - val_actual_accuracy: 0.9080
Epoch 314/800
21s - loss: 0.0070 - actual_accuracy: 0.9847 - val_loss: 0.1071 - val_actual_accuracy: 0.9126
Epoch 315/800
21s - loss: 0.0059 - actual_accuracy: 0.9868 - val_loss: 0.1304 - val_actual_accuracy: 0.8949
Epoch 316/800
21s - loss: 0.0065 - actual_

...

25s - loss: 0.0051 - actual_accuracy: 0.9894 - val_loss: 0.1298 - val_actual_accuracy: 0.9080
Epoch 460/800
23s - loss: 0.0055 - actual_accuracy: 0.9890 - val_loss: 0.1292 - val_actual_accuracy: 0.8969
Epoch 461/800
23s - loss: 0.0046 - actual_accuracy: 0.9901 - val_loss: 0.1349 - val_actual_accuracy: 0.8957
Epoch 462/800
23s - loss: 0.0045 - actual_accuracy: 0.9902 - val_loss: 0.1301 - val_actual_accuracy: 0.9015
Epoch 463/800
24s - loss: 0.0052 - actual_accuracy: 0.9892 - val_loss: 0.1249 - val_actual_accuracy: 0.8949
Epoch 464/800
23s - loss: 0.0047 - actual_accuracy: 0.9902 - val_loss: 0.1261 - val_actual_accuracy: 0.9064
Epoch 465/800
24s - loss: 0.0044 - actual_accuracy: 0.9901 - val_loss: 0.1545 - val_actual_accuracy: 0.8969
Epoch 466/800
24s - loss: 0.0050 - actual_accuracy: 0.9900 - val_loss: 0.1176 - val_actual_accuracy: 0.9034
Epoch 467/800
23s - loss: 0.0045 - actual_accuracy: 0.9893 - val_loss: 0.1257 - val_actual_accuracy: 0.9003
Epoch 468/800
23s - loss: 0.0045 - actual_

23s - loss: 0.0039 - actual_accuracy: 0.9913 - val_loss: 0.1229 - val_actual_accuracy: 0.9107
Epoch 536/800
25s - loss: 0.0040 - actual_accuracy: 0.9916 - val_loss: 0.1477 - val_actual_accuracy: 0.9034
Epoch 537/800
23s - loss: 0.0040 - actual_accuracy: 0.9916 - val_loss: 0.1325 - val_actual_accuracy: 0.9107
Epoch 538/800
23s - loss: 0.0040 - actual_accuracy: 0.9912 - val_loss: 0.1350 - val_actual_accuracy: 0.9003
Epoch 539/800
23s - loss: 0.0040 - actual_accuracy: 0.9917 - val_loss: 0.1375 - val_actual_accuracy: 0.8958
Epoch 540/800
22s - loss: 0.0049 - actual_accuracy: 0.9908 - val_loss: 0.1276 - val_actual_accuracy: 0.8903
Epoch 541/800
23s - loss: 0.0041 - actual_accuracy: 0.9909 - val_loss: 0.1225 - val_actual_accuracy: 0.9015
Epoch 542/800
23s - loss: 0.0043 - actual_accuracy: 0.9909 - val_loss: 0.1462 - val_actual_accuracy: 0.9034
Epoch 543/800
23s - loss: 0.0043 - actual_accuracy: 0.9912 - val_loss: 0.1189 - val_actual_accuracy: 0.9080
Epoch 544/800
22s - loss: 0.0048 - actual_

22s - loss: 0.0039 - actual_accuracy: 0.9919 - val_loss: 0.1357 - val_actual_accuracy: 0.8942
Epoch 612/800
23s - loss: 0.0036 - actual_accuracy: 0.9927 - val_loss: 0.1216 - val_actual_accuracy: 0.9049
Epoch 613/800
23s - loss: 0.0040 - actual_accuracy: 0.9917 - val_loss: 0.1430 - val_actual_accuracy: 0.8992
Epoch 614/800
22s - loss: 0.0042 - actual_accuracy: 0.9910 - val_loss: 0.1291 - val_actual_accuracy: 0.9057
Epoch 615/800
23s - loss: 0.0041 - actual_accuracy: 0.9906 - val_loss: 0.1308 - val_actual_accuracy: 0.8957
Epoch 616/800
24s - loss: 0.0040 - actual_accuracy: 0.9919 - val_loss: 0.1362 - val_actual_accuracy: 0.8962
Epoch 617/800
23s - loss: 0.0038 - actual_accuracy: 0.9921 - val_loss: 0.1411 - val_actual_accuracy: 0.8911
Epoch 618/800
22s - loss: 0.0043 - actual_accuracy: 0.9911 - val_loss: 0.1417 - val_actual_accuracy: 0.8942
Epoch 619/800
23s - loss: 0.0036 - actual_accuracy: 0.9927 - val_loss: 0.1312 - val_actual_accuracy: 0.8976
Epoch 620/800
25s - loss: 0.0045 - actual_

21s - loss: 0.0043 - actual_accuracy: 0.9921 - val_loss: 0.1334 - val_actual_accuracy: 0.9034
Epoch 688/800
22s - loss: 0.0043 - actual_accuracy: 0.9914 - val_loss: 0.1329 - val_actual_accuracy: 0.8984
Epoch 689/800
21s - loss: 0.0035 - actual_accuracy: 0.9929 - val_loss: 0.1232 - val_actual_accuracy: 0.9045
Epoch 690/800
21s - loss: 0.0037 - actual_accuracy: 0.9918 - val_loss: 0.1278 - val_actual_accuracy: 0.9080
Epoch 691/800
21s - loss: 0.0033 - actual_accuracy: 0.9928 - val_loss: 0.1297 - val_actual_accuracy: 0.9003
Epoch 692/800
21s - loss: 0.0038 - actual_accuracy: 0.9925 - val_loss: 0.1235 - val_actual_accuracy: 0.8984
Epoch 693/800
21s - loss: 0.0035 - actual_accuracy: 0.9920 - val_loss: 0.1349 - val_actual_accuracy: 0.8966
Epoch 694/800
21s - loss: 0.0037 - actual_accuracy: 0.9924 - val_loss: 0.1419 - val_actual_accuracy: 0.8949
Epoch 695/800
21s - loss: 0.0038 - actual_accuracy: 0.9916 - val_loss: 0.1313 - val_actual_accuracy: 0.9076
Epoch 696/800
21s - loss: 0.0036 - actual_

21s - loss: 0.0033 - actual_accuracy: 0.9928 - val_loss: 0.1369 - val_actual_accuracy: 0.8957
Epoch 764/800
21s - loss: 0.0032 - actual_accuracy: 0.9931 - val_loss: 0.1447 - val_actual_accuracy: 0.8969
Epoch 765/800
22s - loss: 0.0036 - actual_accuracy: 0.9924 - val_loss: 0.1339 - val_actual_accuracy: 0.9011
Epoch 766/800
21s - loss: 0.0033 - actual_accuracy: 0.9923 - val_loss: 0.1362 - val_actual_accuracy: 0.8930
Epoch 767/800
21s - loss: 0.0031 - actual_accuracy: 0.9935 - val_loss: 0.1406 - val_actual_accuracy: 0.9030
Epoch 768/800
22s - loss: 0.0036 - actual_accuracy: 0.9927 - val_loss: 0.1392 - val_actual_accuracy: 0.9041
Epoch 769/800
22s - loss: 0.0035 - actual_accuracy: 0.9931 - val_loss: 0.1557 - val_actual_accuracy: 0.8896
Epoch 770/800
21s - loss: 0.0039 - actual_accuracy: 0.9924 - val_loss: 0.1227 - val_actual_accuracy: 0.9080
Epoch 771/800
21s - loss: 0.0033 - actual_accuracy: 0.9921 - val_loss: 0.1472 - val_actual_accuracy: 0.8909
Epoch 772/800
22s - loss: 0.0035 - actual_

In [8]:
name = '40_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [7]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()